In [1]:
import macropy.activate
import importlib
import sys
sys.path.append('../')
import torch
import torch.nn as nn
import torch.optim as optim
from Code.iff_macro import set_config
from Code.lstm import lstmPolicyPredictor, FullyConnected
from Code.envs.MountainCar import MultiMountainCar, LookupPolicy, PassiveEnv
from Code.SNN import RSNN, FeedForwardSNN, magicRSNN

In [2]:
config = {
    'ALPHA': 0,
    'BETA': 1,
    'RESET_ZERO': False,
    'THRESH_ADD': 1,
    'THRESH_DECAY': 1,
    'DECODING': 'potential',
    'SPIKE_FN': 'bellec',
    'SIM_TIME': 10
}
set_config(config)
import Code.macroNeurons as Neurons
importlib.reload(Neurons)

<module 'Code.macroNeurons' from '../Code/macroNeurons.py'>

In [3]:
env = PassiveEnv()

In [4]:
BATCH_SIZE = 64

In [12]:
#model = lstmPolicyPredictor(2,8,16)
#model = RSNN(config, 1, 32, 16, 1, Neurons.LIFNeuron, Neurons.AdaptiveNeuron, Neurons.OutputNeuron)
model = magicRSNN(config, 1, 32, 16, 1, Neurons.LIFNeuron, Neurons.MagicNeuron, Neurons.OutputNeuron)
#model = FeedForwardSNN(config, [1, 128, 128, 1], Neurons.LIFNeuron, Neurons.OutputNeuron)
#model = FullyConnected([1, 128, 128, 1])

teacher = LookupPolicy()


#TODO: test superspike instead of Bellec

In [6]:
#bce = nn.BCELoss(reduction='none') #reduction='sum'
mse = nn.MSELoss(reduction='none')
optimizer = optim.Adam(model.parameters(), lr=0.0001)#0.00001

In [7]:
for i in range(5000):
    model.zero_grad()
    inputs, targets, mask = env.getBatch(BATCH_SIZE)
    outputs, _ = model(inputs/0.4)
    #print(outputs.shape, targets.shape)
    loss = (mse(outputs.squeeze(dim=2)/20, targets) * mask).sum() / mask.sum()
    loss.backward()
    optimizer.step()
    if i%10 == 0:
        print(loss.item(), (loss/targets.view(-1).var()).item(), i) #, ((outputs>0.5) != targets).sum()


0.002211832907050848 4.347535610198975 0
0.0019170005107298493 3.624539613723755 10
0.0016767119523137808 3.2489659786224365 20
0.001473296550102532 2.7053163051605225 30
0.0012687068665400147 2.4473164081573486 40
0.0011260226601734757 2.215318202972412 50
0.0009826339082792401 1.8202329874038696 60
0.0008614218095317483 1.7241489887237549 70
0.0007619134848937392 1.4664584398269653 80
0.0006828340701758862 1.2884080410003662 90
0.0006260736845433712 1.1623941659927368 100
0.0005728006362915039 1.1182050704956055 110
0.0005339178605936468 1.035591959953308 120
0.0005053593777120113 1.0022834539413452 130
0.00048571176012046635 0.9366914629936218 140
0.0004604938440024853 0.9241296052932739 150
0.00046307340380735695 0.8620093464851379 160
0.00044172655907459557 0.8777489066123962 170
0.00043628879939205945 0.8639355301856995 180
0.0004310367803554982 0.8423964977264404 190
0.0004301064764149487 0.8468295335769653 200
0.00042926985770463943 0.8063421845436096 210
0.00042852689512073994

KeyboardInterrupt: 

In [ ]:
#torch.save(model, '../models/lstm_passive')
#0.0002

In [ ]:
#torch.save(model, '../models/rsnn_passive_new1')
#0.001

In [9]:
from matplotlib import pyplot as plt
#model = torch.load('../models/snn_passive3')
%matplotlib

Using matplotlib backend: TkAgg


In [ ]:
plt.figure()


In [ ]:
inputs[:, 0, 0].var().sqrt()

In [ ]:
model(inputs/0.4)[0].squeeze()

In [10]:
def doplot():
    inputs, targets, mask = env.getBatch(1)
    outputs, _ = model(inputs/0.4)
    plt.close()
    plt.plot(inputs[:, 0, 0], targets)
    plt.plot(inputs[:, 0, 0], outputs.squeeze().detach()/20)

In [13]:
doplot()